In [5]:
import scapy.all as scapy
import tkinter as tk
from tkinter import ttk
from tkinter import *
from tkinter.ttk import *
from tkinter import scrolledtext as st
import sys
from tkinter import filedialog as fd
from tkinter import messagebox as mb

class Monitor:

    def __init__(self, window):
        # Initializations 
        self.wind = window
        self.wind.title('Network Monitor')
        self.scrolledtext1=st.ScrolledText(self.wind, width=100, height=10)
        self.scrolledtext1.pack(fill=tk.BOTH, side=tk.LEFT, expand=True)
        
    def scan(self,ip):
        self.arp_req_frame = scapy.ARP(pdst = ip)
        self.broadcast_ether_frame = scapy.Ether(dst = "ff:ff:ff:ff:ff:ff")
    
        self.broadcast_ether_arp_req_frame = self.broadcast_ether_frame / self.arp_req_frame

        self.answered_list = scapy.srp(self.broadcast_ether_arp_req_frame, timeout = 1, verbose = False)[0]
        self.result = []
        for i in range(0,len(self.answered_list)):
            self.client_dict = {"ip" : self.answered_list[i][1].psrc, "mac" : self.answered_list[i][1].hwsrc}
            self.result.append(self.client_dict)

        return self.result
    
    def display_result(self,result):
        print("-----------------------------------\nIP Address\tMAC Address\n-----------------------------------")
        for i in self.result:
            print("{}\t{}".format(i["ip"], i["mac"]))
        
if __name__ == '__main__':
    window = tk.Tk()
    application = Monitor(window)
    scanned_output = application.scan('192.168.1.254/24')
    application.display_result(scanned_output)
    window.mainloop()

-----------------------------------
IP Address	MAC Address
-----------------------------------
192.168.1.57	dc:fb:48:66:7d:f8
192.168.1.58	cc:35:40:96:29:f0
192.168.1.52	4c:c9:5e:80:5d:d4
192.168.1.66	b4:86:55:76:ef:c7
192.168.1.254	cc:35:40:96:29:ee
